In [2]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')

import hvplot.pandas

In [3]:
# cache data to improve dashboard performance
if 'data' not in pn.state.cache.keys():

    df = pd.read_csv('data/owid-co2-data.csv')

    pn.state.cache['data'] = df.copy()

else: 

    df = pn.state.cache['data']

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50598 entries, 0 to 50597
Data columns (total 79 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   country                                    50598 non-null  object 
 1   year                                       50598 non-null  int64  
 2   iso_code                                   41970 non-null  object 
 3   population                                 40008 non-null  float64
 4   gdp                                        14564 non-null  float64
 5   cement_co2                                 24974 non-null  float64
 6   cement_co2_per_capita                      22714 non-null  float64
 7   co2                                        31349 non-null  float64
 8   co2_growth_abs                             28944 non-null  float64
 9   co2_growth_prct                            25032 non-null  float64
 10  co2_including_luc     

In [5]:
df[df['country'] == 'Germany']

,country,year,iso_code,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_growth_abs,co2_growth_prct,...,share_global_other_co2,share_of_temperature_change_from_ghg,temperature_change_from_ch4,temperature_change_from_co2,temperature_change_from_ghg,temperature_change_from_n2o,total_ghg,total_ghg_excluding_lucf,trade_co2,trade_co2_share
17722,Germany,1792,DEU,NaN,NaN,NaN,NaN,0.469,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17723,Germany,1793,DEU,NaN,NaN,NaN,NaN,0.480,0.011,2.344,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17724,Germany,1794,DEU,NaN,NaN,NaN,NaN,0.443,-0.037,-7.634,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17725,Germany,1795,DEU,NaN,NaN,NaN,NaN,0.447,0.004,0.826,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17726,Germany,1796,DEU,NaN,NaN,NaN,NaN,0.535,0.088,19.672,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17947,Germany,2017,DEU,82624368.0,3.827498e+12,13.408,0.162,785.616,-14.723,-1.840,...,2.840,3.027,0.004,0.039,0.046,0.002,800.21,829.68,113.953,14.505
17948,Germany,2018,DEU,82896696.0,3.885961e+12,13.228,0.160,754.408,-31.208,-3.972,...,2.885,2.995,0.004,0.040,0.046,0.002,771.48,800.96,130.370,17.281
17949,Germany,2019,DEU,83148144.0,NaN,13.287,0.160,707.150,-47.258,-6.264,...,2.726,2.962,0.004,0.040,0.046,0.002,720.23,749.71,135.700,19.190
17950,Germany,2020,DEU,83328992.0,NaN,13.357,0.160,639.381,-67.769,-9.583,...,2.646,2.930,0.004,0.040,0.046,0.002,NaN,NaN,130.111,20.350


## (0) Data preprocessing

In [6]:
# Fill NAs with 0s
df = df.fillna(0)
# Create GDP per capita column
df['gdp_per_capita'] = np.where(df['population']!= 0, df['gdp']/ df['population'], 0)

In [7]:
# Make DataFrame Pipeline Interactive
idf = df.interactive()

## (1) Historische CO2-Emissionen nach Kontinent

In [8]:
# Create Panel widgets: Year Slider
year_slider = pn.widgets.IntSlider(name='Jahr', start=1750, end=2020, step=5, value=1850)
year_slider

IntSlider(end=2020, name='Jahr', start=1750, step=5, value=1850)

In [9]:
# Create Panel widgets: Radio buttons für CO2
yaxis_co2 = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['co2', 'co2_per_capita',],
    button_type='primary'
)
yaxis_co2

RadioButtonGroup(button_type='primary', name='Y axis', options=['co2', 'co2_per_capita'], value='co2')

In [10]:
continents = ['World', 'Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica']

co2_pipeline = (
    idf[
        (idf.year <= year_slider) &
        (idf.country.isin(continents))
    ]
    .groupby(['country', 'year'])[yaxis_co2].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year')  
    .reset_index(drop=True)
)

In [11]:
co2_pipeline

In [12]:
co2_plot = co2_pipeline.hvplot(x = 'year', by='country', y=yaxis_co2,line_width=2, title="CO2-Emissionen nach Kontinent")
co2_plot

## (2) Table - Historische CO2-Emission nach Kontinent

In [13]:
co2_table = co2_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width') 
co2_table

## (3) CO2 vs BIP Scatterplot

In [14]:
co2_vs_gdp_scatterplot_pipeline = (
    idf[
        (idf.year == year_slider) &
        (~ (idf.country.isin(continents)))
    ]
    .groupby(['country', 'year', 'gdp_per_capita'])['co2'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year')  
    .reset_index(drop=True)
)

In [15]:
co2_vs_gdp_scatterplot_pipeline

In [16]:
co2_vs_gdp_scatterplot = co2_vs_gdp_scatterplot_pipeline.hvplot(x='gdp_per_capita', 
                                                                y='co2', 
                                                                by='country', 
                                                                size=80, kind="scatter", 
                                                                alpha=0.7,
                                                                legend=False, 
                                                                height=500, 
                                                                width=500)
co2_vs_gdp_scatterplot

## (4) Balkendiagramm mit der CO2 Sources nach Kontinent

In [17]:
yaxis_co2_source = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['coal_co2', 'oil_co2', 'gas_co2'], 
    button_type='danger'
)

continents_excl_world = ['Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica']

co2_source_bar_pipeline = (
    idf[
        (idf.year == year_slider) &
        (idf.country.isin(continents_excl_world))
    ]
    .groupby(['year', 'country'])[yaxis_co2_source].sum()
    .to_frame()
    .reset_index()
    .sort_values(by='year')  
    .reset_index(drop=True)
)

In [18]:
co2_source_bar_plot = co2_source_bar_pipeline.hvplot(kind='bar', 
                                                     x='country', 
                                                     y=yaxis_co2_source, 
                                                     title='CO2-Ursprung nach Kontinent')
co2_source_bar_plot

## Creating Dashboard

In [19]:
# Layout using Template
template = pn.template.FastListTemplate(
    title='Historische globale CO2-Emissionen Dashboard', 
    sidebar=[pn.pane.Markdown("# CO2 Emissionen und Klimawandel"), 
             pn.pane.Markdown("#### Kohlendioxidemissionen sind die Hauptursache für den globalen Klimawandel. Es ist allgemein anerkannt, dass die Welt die Emissionen dringend reduzieren muss, um die schlimmsten Auswirkungen des Klimawandels zu vermeiden. Doch wie diese Verantwortung zwischen Regionen, Ländern und Einzelpersonen aufgeteilt wird, ist ein ständiger Streitpunkt in internationalen Diskussionen."), 
             pn.pane.PNG('img/climate_day.png', sizing_mode='scale_both'),
             pn.pane.Markdown("## Einstellungen"),   
             year_slider],
    main=[pn.Row(pn.Column(yaxis_co2, 
                           co2_plot.panel(width=700), margin=(0,25)), 
                 co2_table.panel(width=500)), 
          pn.Row(pn.Column(co2_vs_gdp_scatterplot.panel(width=600), margin=(0,25)), 
                 pn.Column(yaxis_co2_source, co2_source_bar_plot.panel(width=600)))],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)
# template.show()
template.servable();